<a href="https://colab.research.google.com/github/Renmsd/portfoilo/blob/main/Gen%20Ai/LLM/RAG/Copy_of_RAG_Conquering_the_Last_Mile(GOOGLE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG: Conquering the Last Mile


**Workshop Objective:** Navigating the final steps from a working prototype to a production service is often the hardest part of the journey. This workshop is your guide for conquering "the last mile" of RAG development. You will learn the essential techniques to **validate, secure, and monitor** your RAG application to make it reliable and production-ready.

**Our Journey:**
1.  **The Prototype:** We'll quickly build a working RAG system using a common knowledge dataset (Planets of the Solar System).
2.  **The Last Mile - Step 1: VALIDATE:** We'll programmatically evaluate our prototype's performance using the Vertex AI Evaluation Service to get baseline quality metrics. *Is our prototype actually any good?*
3.  **The Last Mile - Step 2: SECURE:** We'll add a critical production feature: access control. We will modify our RAG system to respect user permissions and only return data they are authorized to see. *Is our system safe to use?*
4.  **The Last Mile - Step 3: MONITOR:** We'll implement logging to capture a record of every query, its result, and user feedback. This is the foundation for monitoring system health and performance over time. *Do we know how our system is performing in the wild?*
5.  **Closing the Loop (TUNE & RE-VALIDATE):** Using our validation framework, we'll tune a key parameter (chunk size), rebuild our system, and re-evaluate to measure the impact of our change, demonstrating a data-driven approach to improvement.


Workshop Slides  : https://docs.google.com/presentation/d/1w5aLc3aopumFoLeyX7J8hFH3Pb4gFPlyaJMh13hNGzA/edit?usp=sharing



## Step 0: Setup and Authentication

First, we'll authenticate our Colab environment to access our Google Cloud project and install the required Python libraries.

In [ ]:
print("Installing - 1")
!pip install -q google-cloud-aiplatform[evaluation]
print("Installing - 2 ")
!pip install -q --upgrade google-colab google-cloud-bigquery google-cloud-aiplatform langchain "langchain-google-vertexai>1.0.0"
# pandas pyarrow
print("Installing - 3 ")
!pip install -q --upgrade pandas pyarrow
print("Installing - 4 ")
!pip install -q google-cloud-aiplatform[evaluation] ragas rouge_score
# Restart runtime to load the newly installed package
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Installing - 1
Installing - 2 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.31 requires requests<3.0.0,>=2.32.5, but you have requests 2.32.4 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
Installing - 3 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
dask-cudf-cu12 25.6.0 requires 

{'status': 'ok', 'restart': True}

In [ ]:
# # # # # # # # # # # # #
# Run on Restart.       #
# # # # # # # # # # # #

from google.colab import auth
import os



auth.authenticate_user()

# @title Configure GCP Project Details
PROJECT_ID = 'platinum-bebop-474816-p0' # @param {type:"string"}
DATASET = 'dataset'     # @param {type:"string"}
LOCATION = 'us-central1'        # Use us-central1

# Set the project ID for the gcloud command-line tool and Python libraries.
os.environ['GCLOUD_PROJECT'] = PROJECT_ID

print(f"Project ID set to: {PROJECT_ID}")
print(f"BigQuery Dataset to be used/created: {DATASET}")






Project ID set to: platinum-bebop-474816-p0
BigQuery Dataset to be used/created: dataset





Define your models

In [ ]:
# # # # # # # # # # # # #
# Run on Restart. #
# # # # # # # # # # # #

from google.cloud import bigquery
bq_client = bigquery.Client(project=PROJECT_ID)

EMBEDDING_MODEL = "text-embedding-005"
MODEL_ID="gemini-2.0-flash"

In [ ]:
!gcloud services enable aiplatform.googleapis.com --project {PROJECT_ID}


Operation "operations/acat.p2-1011251704725-84d2e9df-b66c-4839-8e0b-751f971c504d" finished successfully.


## Step 1: The Prototype - Create the Source Data Table and Define Remote Model

We will start by curating our knowledge base. We will create a small, focused table in our own BigQuery dataset containing just the articles for the eight planets in our solar system. This is our "source of truth."

We define the embedding model as a Remote Model to be able to use for vectorizing our data

In [ ]:
CONN_ID="demo_connection"
!bq mk --connection --location={LOCATION} --project_id={PROJECT_ID} \
  --connection_type=CLOUD_RESOURCE {CONN_ID}

!bq show --connection {PROJECT_ID}.{LOCATION}.{CONN_ID}

Connection 1011251704725.us-central1.demo_connection successfully created
Connection platinum-bebop-474816-p0.us-central1.demo_connection

                    name                      friendlyName   description    Last modified         type        hasCredential                                             properties                                            
 ------------------------------------------- -------------- ------------- ----------------- ---------------- --------------- ------------------------------------------------------------------------------------------------ 
  1011251704725.us-central1.demo_connection                                11 Oct 16:36:26   CLOUD_RESOURCE   False           {"serviceAccountId": "bqcx-1011251704725-xcx2@gcp-sa-bigquery-condel.iam.gserviceaccount.com"}  



In [ ]:
# Copy the service account name from above command
SERV_ACCOUNT="bqcx-1011251704725-xcx2@gcp-sa-bigquery-condel.iam.gserviceaccount.com"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
  --member="serviceAccount:{SERV_ACCOUNT}" \
  --role="roles/aiplatform.user"

Updated IAM policy for project [platinum-bebop-474816-p0].
bindings:
- members:
  - serviceAccount:bqcx-1011251704725-xcx2@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  role: roles/aiplatform.user
- members:
  - user:Renmsd@gmail.com
  role: roles/owner
etag: BwZA5KbfgZw=
version: 1


In [ ]:


# Create the dataset if it doesn't exist
dataset_id = f"{PROJECT_ID}.{DATASET}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = LOCATION
bq_client.create_dataset(dataset, exists_ok=True)
print(f"Dataset '{dataset_id}' is ready.")

create_source_table_sql = f"""
CREATE OR REPLACE TABLE `{dataset_id}.wikipedia_planets` AS
SELECT
  page_title as title,
  summary as text
FROM
  `bqaiqtest.rag_demo_dataset.wikipedia_planets_source`

"""

print("Creating curated source table from public Wikipedia data...")
job = bq_client.query(create_source_table_sql)
job.result() # Wait for the job to complete
print(f"Table `{dataset_id}.wikipedia_planets` created successfully.")


# Create the remote text embedding model to use

create_remote_model_sql = f"""
CREATE OR REPLACE MODEL `{dataset_id}.text_embedding_model`
REMOTE WITH CONNECTION DEFAULT
OPTIONS(ENDPOINT = '{EMBEDDING_MODEL}');
"""
print("Creating remote text embedding model")

job = bq_client.query(create_remote_model_sql)
job.result() # Wait for the job to complete
print(f"Remote model `{dataset_id}.text_embedding_model` created successfully.")



Dataset 'platinum-bebop-474816-p0.dataset' is ready.
Creating curated source table from public Wikipedia data...
Table `platinum-bebop-474816-p0.dataset.wikipedia_planets` created successfully.
Creating remote text embedding model
Remote model `platinum-bebop-474816-p0.dataset.text_embedding_model` created successfully.


## Step 2: The Prototype - Build the RAG Knowledge Base

Here, we build our initial RAG store. To simulate a real-world scenario, we'll add an `access_level` column to our data, marking some documents as 'Internal' and some as 'Public'. This will be crucial for the 'Secure' step later.

In [ ]:
# # # # # # # # # # # # #
# Run on Restart. #
# # # # # # # # # # # #

import uuid
import random
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from google.cloud.bigquery import ScalarQueryParameter
from google.cloud.bigquery.job import LoadJobConfig
from google.cloud.bigquery.table import Table



def build_rag_store(rag_table_name: str, chunk_size: int, chunk_overlap: int):
    """Reads from the source BQ table, chunks text, creates embeddings using BigQuery, and loads to a new RAG table."""

    dataset_id = f"{PROJECT_ID}.{DATASET}"
    source_table_id = f"{dataset_id}.wikipedia_planets"
    destination_table_id = f"{dataset_id}.{rag_table_name}"
    temp_chunked_table_id = f"{dataset_id}.temp_chunked_{rag_table_name}"


    # 1. Create the destination RAG table schema with an access_level column
    create_rag_table_sql = f"""
    CREATE OR REPLACE TABLE `{destination_table_id}` (
      chunk_id STRING NOT NULL,
      source_title STRING,
      content STRING,
      access_level STRING,
      embedding ARRAY<FLOAT64>
    );
    """
    print(f"Creating RAG table: {destination_table_id}")
    bq_client.query(create_rag_table_sql).result()

    # 2. Read source data and chunk documents
    print(f"Reading data from {source_table_id} and chunking...")
    # Workaround for pyarrow/pandas compatibility issue: Fetch results as list of dicts
    query_job = bq_client.query(f"SELECT title, text FROM `{source_table_id}`")
    source_data = [dict(row) for row in query_job.result()]
    source_df = pd.DataFrame(source_data)


    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )

    rows_to_insert_temp = []
    for _, row in source_df.iterrows():
        chunks = text_splitter.split_text(row['text'])
        for chunk in chunks:
            rows_to_insert_temp.append({
                "chunk_id": str(uuid.uuid4()),
                "source_title": row['title'],
                "content": chunk,
                 # Simulate a mix of public and internal documents
                "access_level": random.choice(['Public', 'Internal'])
            })
    print(f"Generated a total of {len(rows_to_insert_temp)} chunks.")

    # 3. Load chunked data to a temporary BigQuery table
    print(f"Loading chunked data to a temporary BigQuery table: {temp_chunked_table_id}")
    # Define the schema for the temporary table
    temp_table_schema = [
        bigquery.SchemaField("chunk_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("source_title", "STRING"),
        bigquery.SchemaField("content", "STRING"),
        bigquery.SchemaField("access_level", "STRING"),
    ]
    temp_table = Table(temp_chunked_table_id, schema=temp_table_schema)

    # Create the temporary table, ensuring it exists before inserting
    bq_client.create_table(temp_table, exists_ok=True)

    errors = bq_client.insert_rows_json(temp_chunked_table_id, rows_to_insert_temp)
    if errors:
        print(f"Encountered errors loading to temporary table: {errors}")
        return # Exit if loading to temp table fails

    # 4. Generate embeddings using BigQuery ML
    print(f"Generating embeddings using BigQuery ML ({EMBEDDING_MODEL})...")
    generate_embeddings_sql = f"""
    INSERT INTO `{destination_table_id}` (chunk_id, source_title, content, access_level, embedding)
    SELECT
        chunk_id,
        source_title,
        content,
        access_level,
        ml_generate_embedding_result
    FROM
        ML.GENERATE_EMBEDDING(
            MODEL `{dataset_id}.text_embedding_model`,
            (SELECT chunk_id, source_title, content, access_level FROM `{temp_chunked_table_id}`)
        )
    """
    # print(generate_embeddings_sql)
    job = bq_client.query(generate_embeddings_sql)
    job.result() # Wait for the job to complete
    print(f"Successfully generated embeddings and loaded to {destination_table_id}")

    # Clean up the temporary table
    bq_client.query(f"DROP TABLE `{temp_chunked_table_id}`").result()
    print(f"Temporary table {temp_chunked_table_id} dropped.")


BASELINE_RAG_TABLE = "planets_rag_baseline"


In [ ]:

# --- Build our first, baseline RAG store ---
build_rag_store(
    rag_table_name=BASELINE_RAG_TABLE,
    chunk_size=1000,
    chunk_overlap=100
)

## Step 3: The Prototype - Create a Vector Index for Fast Retrieval

To make our similarity searches fast, we create a vector index. This is a crucial step for ensuring a good user experience in a production application.

In [ ]:
# # # # # # # # # # # # #
# Run on Restart. #
# # # # # # # # # # # #





def create_vector_index(rag_table_name: str):
    index_name = f"{rag_table_name}_embedding_index"
    table_id = f"{PROJECT_ID}.{DATASET}.{rag_table_name}"

    sql = f"""
    CREATE OR REPLACE VECTOR INDEX `{index_name}`
    ON `{table_id}`(embedding)
    OPTIONS(index_type='IVF', distance_type='COSINE');
    """
    print(f"Creating vector index `{index_name}` on table `{table_id}`. This may take a few minutes...")
    print(f"If you have less than 5000 rows, an error is returned. Ignore it and go to next step")

    job = bq_client.query(sql)
    job.result()
    print("Vector index created successfully.")



In [ ]:
# Create the index for our baseline table


create_vector_index(BASELINE_RAG_TABLE)

## Step 4: The Last Mile - SECURE Your RAG Pipeline

A prototype often ignores security, but a production system cannot. We will now upgrade our RAG pipeline to be 'security-aware'. It will take a `user_access_level` and filter the retrieved documents to ensure a user can only see 'Public' information or information matching their access level.

We achieve this by adding a `filter` option to the `VECTOR_SEARCH` function, which is the most efficient way to apply a metadata filter *before* the vector search runs.

In [ ]:

# # # # # # # # # # # # #
# Run on Restart. #
# # # # # # # # # # # #


from langchain_google_vertexai import VertexAI
from langchain_google_vertexai import VertexAIEmbeddings


GENERATION_MODEL = MODEL_ID
llm = VertexAI(model_name=GENERATION_MODEL, project=PROJECT_ID)
embeddings_service = VertexAIEmbeddings(model_name=EMBEDDING_MODEL, project=PROJECT_ID)

def ask_rag_system(question: str, rag_table_name: str, user_access_level: str = 'Public') -> dict:
    """Combines SECURE retrieval and generation to answer a question."""
    table_id = f"{PROJECT_ID}.{DATASET}.{rag_table_name}"

    # 1. Generate an embedding for the user's question
    question_embedding = embeddings_service.embed_query(question)

    # 2. Retrieve relevant chunks using a SECURE VECTOR_SEARCH
    # We filter to only include chunks that are 'Public' or match the user's access level.
    security_filter = f"access_level = 'Public' OR access_level = '{user_access_level}'"
    security_filter = f"access_level = '{user_access_level}'"
    # print(security_filter)

    security_filter_clause="WHERE " f"{security_filter}"
    retrieve_query = f"""
    SELECT
        base.content, base.access_level
    FROM
        VECTOR_SEARCH(
             ( SELECT * FROM `{table_id}` {security_filter_clause}) ,
            'embedding',
            (SELECT {question_embedding} AS embedding),
            top_k => 3,
            distance_type => 'COSINE'
        )
    """
    # print(retrieve_query)
    results = bq_client.query(retrieve_query).result()
    context_chunks = [row.content for row in results]
    context_str = "\n\n".join(context_chunks)

    # 3. Generate the final answer
    prompt = f"""
    Based ONLY on the following context, answer the user's question.
    If the context does not contain the answer, say "The provided context does not have the answer."

    Context:
    {context_str}

    Question:
    {question}

    Answer:
    """
    answer = llm.invoke(prompt)

    # The dictionary returned will be used for both logging and evaluation.
    return {
        "question": question,
        "predicted_answer": answer,
        "context": context_str,
        "user_access_level": user_access_level
    }



In [ ]:
# --- Test the security feature ---
test_question = "Describe the atmosphere of Venus."
print("--- Querying as a 'Public' user ---")
public_result = ask_rag_system(test_question, BASELINE_RAG_TABLE, user_access_level='Public')
print(f"Answer for Public User:\n{public_result['predicted_answer']}")

print("\n--- Querying as an 'Internal' user ---")
internal_result = ask_rag_system(test_question, BASELINE_RAG_TABLE, user_access_level='Internal')
print(f"Answer for Internal User:\n{internal_result['predicted_answer']}")
print("\nNote: The answers may differ if the retrieval process found different context due to access levels.")

## Step 5: The Last Mile - MONITOR Your RAG System

A production system that isn't monitored is a black box. We need to log interactions to understand how our system is being used, where it's failing, and how to improve it.

We'll create a simple logging table in BigQuery and a function to record each interaction. For this demo, we'll simulate a user feedback score.

In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client(project=PROJECT_ID)


LOGGING_TABLE = f"{PROJECT_ID}.{DATASET}.rag_monitoring_logs"

# 1. Create the logging table
create_logging_table_sql = f"""
CREATE TABLE IF NOT EXISTS  `{LOGGING_TABLE}` (
    timestamp TIMESTAMP,
    question STRING,
    predicted_answer STRING,
    context STRING,
    user_access_level STRING,
    simulated_feedback_score INT64 -- e.g., 1 to 5
);
"""
print(f"Creating logging table: {LOGGING_TABLE}")
bq_client.query(create_logging_table_sql).result()
# time.sleep(30)

# 2. Create a logging function
def log_rag_interaction(rag_result: dict):
    """Writes a record of the RAG interaction to a BigQuery logging table."""
    from datetime import datetime

    log_entry = {
        "timestamp": datetime.utcnow().isoformat(),
        "question": rag_result['question'],
        "predicted_answer": rag_result['predicted_answer'],
        "context": rag_result['context'],
        "user_access_level": rag_result['user_access_level'],
        "simulated_feedback_score": random.randint(3, 5) # Simulate good user feedback
    }

    errors = bq_client.insert_rows_json(LOGGING_TABLE, [log_entry])
    if errors:
        print(f"Failed to log interaction: {errors}")

# 3. Test the logging
print("Running a test query to generate a log entry...")
test_log_result = ask_rag_system("Which planet is known as the Red Planet?", BASELINE_RAG_TABLE)
log_rag_interaction(test_log_result)
print("Log entry created.")

# 4. Query the logs to show how monitoring would work
print("\n--- Sample query on monitoring logs --- ")
log_query_df = bq_client.query(f"SELECT timestamp, question, simulated_feedback_score FROM `{LOGGING_TABLE}` ORDER BY timestamp DESC LIMIT 5").to_dataframe()
print(log_query_df)
display(log_query_df)


## Step 6: The Last Mile - VALIDATE RAG Performance

With our secure and monitored pipeline in place, we now validate its quality. We will use the **Vertex AI Evaluation Service** to systematically measure performance.

We will calculate  key metrics:
1.  **Groundedness:** Does the generated answer stay faithful to the retrieved context? (Measures hallucinations)
2.  **Accuracy:** Does it provide accurate information consistent with the retrieved context
3. **Completeness:**


First Define the evaluation datasets.

The evaluation datasets contains
 - Query/Prompt - The prompt for which we are evaluating.
 - Context - This is obtained when we run the prompt against the vector database. These are the vectors/text chunks passed to the LLM for response generation
 - Response - The respone from the LLM
 - Ground Truth - Depending on the metric, we may provide the exepected output to compare against the LLM Generated response

 Here we are creating two datasets: eval_dataset_small and eval_dataset_large
 You can use just one or both

In [ ]:

# # # # # # # # # # # # #
# Run on Restart. #
# # # # # # # # # # # #

from tqdm import tqdm
evaluation_dataset = pd.DataFrame({
    "question": [
        "What is the Great Red Spot on Jupiter?",
        "What are Saturn's rings primarily made of?",
        "Which planet is known as the Red Planet?",
        "Describe the atmosphere of Venus."
    ],
    "ground_truth_answer": [
        "The Great Red Spot is a persistent high-pressure region in the atmosphere of Jupiter, producing an anticyclonic storm that is the largest in the Solar System.",
        "The rings consist of countless small particles, ranging in size from micrometers to meters, that orbit Saturn. The particles are made almost entirely of water ice, with a trace component of rocky material.",
        "Mars is known as the Red Planet due to the prevalence of iron oxide on its surface, which gives it a reddish appearance.",
        "Venus has an extremely dense atmosphere composed of more than 96% carbon dioxide, with the rest being nitrogen and trace gases. The atmosphere traps heat, creating a runaway greenhouse effect resulting in surface temperatures of around 462 °C (864 °F)."
    ]
})


rag_retrieved_context = []
rag_generated_response = []
rag_prompt=[]

for prompt in tqdm(evaluation_dataset['question']):

   rag_result = ask_rag_system(prompt, BASELINE_RAG_TABLE, user_access_level='Public')
   rag_prompt.append(rag_result['question'])
   rag_retrieved_context.append(rag_result['context'])
   rag_generated_response.append(rag_result['predicted_answer'])


eval_dataset_small = pd.DataFrame(
    {
        "prompt": rag_prompt,
        "retrieved_context": rag_retrieved_context,
        "response": rag_generated_response,
    }
)

eval_dataset_small



In [ ]:

# # # # # # # # # # # # #
# Run on Restart. #
# # # # # # # # # # # #


#  This creates a larger dataset for evaluation. (optional)

questions =  [


"Which planet is known as the Red Planet?",

"What are the rings of Saturn primarily made of?",

"How long is a year on Mercury in Earth days?",

"Which planet is the smallest in our solar system?",

"What is the name of the largest volcano on Mars?",

"Which planet is the windiest in the solar system?",

"What is the main component of Venus's atmosphere?",

"Who is the planet Neptune named after?",



"Describe the Great Red Spot on Jupiter.",

"Explain the greenhouse effect that occurs on Venus.",

"Summarize the key characteristics of Earth's magnetosphere.",

"What is the composition of Jupiter's atmosphere?",

"Describe the physical appearance and color of Uranus.",

"Explain why Mercury experiences such extreme temperature fluctuations.",

"What does the article say about the axial tilt of Uranus?",



"Compare the atmospheric composition of Earth and Mars.",

"Which planet is larger in diameter, Uranus or Neptune?",

"Contrast the surface conditions of Venus and Mercury.",

"How does the length of a day on Jupiter compare to a day on Earth?",

"Which of the two ice giants, Uranus or Neptune, has a stronger magnetic field according to the texts?",

"Compare the number of known moons for Saturn and Jupiter.",

"Which of the terrestrial planets has the thinnest atmosphere?",



"Does the article on Venus mention the presence of volcanoes?",

"Is there any mention of liquid water currently on the surface of Mars in the provided text?",

"Do the articles mention any planned future missions to Uranus?",
 "What is the colour of my car?"
  ]

rag_retrieved_context = []
rag_generated_response = []
rag_prompt=[]

for prompt in tqdm(questions):

   rag_result = ask_rag_system(prompt, BASELINE_RAG_TABLE, user_access_level='Public')
   rag_prompt.append(rag_result['question'])
   rag_retrieved_context.append(rag_result['context'])
   rag_generated_response.append(rag_result['predicted_answer'])


eval_dataset_large = pd.DataFrame(
    {
        "prompt": rag_prompt,
        "retrieved_context": rag_retrieved_context,
        "response": rag_generated_response,
    }
)

eval_dataset_large[0:4]


In [ ]:
# # # # # # # # # # # # #
# Run on Restart. #
# # # # # # # # # # # #

from vertexai.evaluation import (
    EvalTask,
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
    notebook_utils,
)

custom_question_answering_correctness = PointwiseMetric(
    metric="custom_question_answering_correctness",
    metric_prompt_template=PointwiseMetricPromptTemplate(
        criteria={
            "accuracy": (
                "The response provides completely accurate information, consistent with the retrieved context, with no errors or omissions."
            ),
            "completeness": (
                "The response answers all parts of the question fully, utilizing the information available in the retrieved context."
            ),
            "groundedness": (
                "The response uses only the information provided in the retrieved context and does not introduce any external information or hallucinations."
            ),
        },
        rating_rubric={
            "5": "(Very good). The answer is completely accurate, complete, concise, grounded in the retrieved context, and follows all instructions.",
            "4": "(Good). The answer is mostly accurate, complete, and grounded in the retrieved context, with minor issues in conciseness or instruction following.",
            "3": "(Ok). The answer is partially accurate and complete but may have some inaccuracies, omissions, or significant issues with conciseness, groundedness, or instruction following, based on the retrieved context.",
            "2": "(Bad). The answer contains significant inaccuracies, is largely incomplete, or fails to follow key instructions, considering the information available in the retrieved context.",
            "1": "(Very bad). The answer is completely inaccurate, irrelevant, or fails to address the question in any meaningful way, based on the retrieved context.",
        },
        input_variables=["prompt", "retrieved_context"],
    ),
)

accuracy_metric = PointwiseMetric(
    metric="accuracy",
    metric_prompt_template=PointwiseMetricPromptTemplate(
        criteria={
            "accuracy": (
                "The response provides completely accurate information, consistent with the retrieved context, with no errors or omissions."
            )
        },
        rating_rubric={
            "5": "(Very good). The answer is completely accuratein the retrieved context, and follows all instructions.",
            "4": "(Good). The answer is mostly accurate in the retrieved context, with minor issues in conciseness or instruction following.",
            "3": "(Ok). The answer is partially accurate based on the retrieved context",
            "2": "(Bad). The answer contains significant inaccuracies",
            "1": "(Very bad). The answer is completely inaccurate, irrelevant, or fails to address the question in any meaningful way, based on the retrieved context.",
        },
        input_variables=["prompt", "retrieved_context"],
    ),
)


completeness_metric = PointwiseMetric(
    metric="completeness",
    metric_prompt_template=PointwiseMetricPromptTemplate(
        criteria={
              "completeness": (
                "The response answers all parts of the question fully, utilizing the information available in the retrieved context."
            ),
        },
        rating_rubric={
            "5": "(Very good). The answer is  complete, concise and follows all instructions.",
            "4": "(Good). The answer is mostly complete with minor issues in conciseness or instruction following.",
            "3": "(Ok). The answer is partially  complete based on the retrieved context.",
            "2": "(Bad). The answer  is largely incomplete, or fails to follow key instructions, considering the information available in the retrieved context.",
            "1": "(Very bad). The answer fails to address the question in any meaningful way, based on the retrieved context.",
        },
        input_variables=["prompt", "retrieved_context"],
    ),
)

groundedness_metric = PointwiseMetric(
    metric="groundedness",
    metric_prompt_template=PointwiseMetricPromptTemplate(
        criteria={

            "groundedness": (
                "The response uses only the information provided in the retrieved context and does not introduce any external information or hallucinations."
            ),
        },
        rating_rubric={
            "5": "(Very good). The answer is completely grounded in the retrieved context, and follows all instructions.",
            "4": "(Good). The answer is mostly grounded in the retrieved context, with minor issues in conciseness or instruction following.",
            "3": "(Ok). The answer is partially grounded  based on the retrieved context.",
            "2": "(Bad). The answer contains significant differences considering the information available in the retrieved context.",
            "1": "(Very bad). The answer fails to address the question in any meaningful way, based on the retrieved context.",
        },
        input_variables=["prompt", "retrieved_context"],
    ),
)

# Display the serialized metric prompt template
# print(custom_question_answering_correctness.metric_prompt_template)

print(completeness_metric.metric_prompt_template)






In [ ]:

# # # # # # # # # # # # #
# Run on Restart.       #
# # # # # # # # # # # #

def run_evaluation(evaluation_ds, metrics_list, experiment_name):
  eval_task = EvalTask(
      dataset=evaluation_ds,
      metrics=metrics_list,
      experiment=experiment_name,
  )
  return eval_task.evaluate()


def print_eval_results(eval_result, metrics_list):

  notebook_utils.display_eval_result(eval_result=eval_result)

  notebook_utils.display_radar_plot(
            eval_results_with_title=[("Question answering correctness", eval_result)],
            metrics = metrics_list
  )

  notebook_utils.display_bar_plot(
            eval_results_with_title=[("Question answering correctness", eval_result)],
            metrics=metrics_list,
        )

  notebook_utils.display_explanations(eval_result=eval_result, num=1)






In [ ]:


base_eval_result_small_ds=run_evaluation(eval_dataset_small, [accuracy_metric, completeness_metric, groundedness_metric], "base-evaluation-small-ds")

print_eval_results(base_eval_result_small_ds, ["accuracy","completeness","groundedness"] )

base_eval_small_ds_summary=base_eval_result_small_ds.summary_metrics


In [ ]:
base_eval_result_large_ds=run_evaluation(eval_dataset_large, [accuracy_metric, completeness_metric, groundedness_metric], "base-evaluation-large-ds")

print_eval_results(base_eval_result_large_ds, ["accuracy","completeness","groundedness"] )

base_eval_large_ds_summary=base_eval_result_large_ds.summary_metrics


## Step 7: Closing the Loop - TUNE and Re-Evaluate

Our baseline system has a set of scores. Can we do better? Let's hypothesize that a smaller chunk size might provide more focused context to the LLM and improve performance.

This is the core loop of production RAG development: **Validate -> Tune -> Re-Validate**.

In [ ]:
# --- 1. Build the TUNED RAG store with smaller chunks ---
TUNED_RAG_TABLE = "planets_rag_tuned"

build_rag_store(
    rag_table_name=TUNED_RAG_TABLE,
    chunk_size=250, # Drastically smaller chunk size
    chunk_overlap=25
)

# --- 2. Create a vector index for the new table ---  (Run if table is over 5000 rows)
# create_vector_index(TUNED_RAG_TABLE)

# --- 3. Evaluate the TUNED model ---
print("\nRunning evaluation for the TUNED RAG system (chunk_size=250)...")
tuned_eval_result_large_ds=run_evaluation(eval_dataset_large, [accuracy_metric, completeness_metric, groundedness_metric], "tuned-evaluation-large-ds")
# print_eval_results(tuned_eval_result_large_ds, ["accuracy","completeness","groundedness"] )
tuned_eval_large_ds_summary=tuned_eval_result_large_ds.summary_metrics

tuned_eval_result_small_ds=run_evaluation(eval_dataset_small, [accuracy_metric, completeness_metric, groundedness_metric], "tuned-evaluation-small-ds")
# print_eval_results(tuned_eval_result_small_ds, ["accuracy","completeness","groundedness"] )
tuned_eval_small_ds_summary=tuned_eval_result_small_ds.summary_metrics



## Step 8: Final Analysis - Conquering the Last Mile

Now we compare the metrics from our two experiments. This is the payoff—we have concrete data to support our tuning decisions, turning guesswork into engineering.

In [ ]:
print("--- Final Comparison of Large Dataset---")
print(f"Baseline (chunk_size=1000): Accuracy={base_eval_large_ds_summary['accuracy/mean']:.4f}, Completeness={base_eval_large_ds_summary['completeness/mean']:.4f}, Groundedness={base_eval_large_ds_summary['groundedness/mean']:.4f}")
print(f"Tuned    (chunk_size=250): Accuracy={tuned_eval_large_ds_summary['accuracy/mean']:.4f}, Completeness={tuned_eval_large_ds_summary['completeness/mean']:.4f}, , Groundedness={tuned_eval_large_ds_summary['groundedness/mean']:.4f}")
print("------------------------")

notebook_utils.display_bar_plot(
    eval_results_with_title=[
        ("Baseline (chunk_size=1000)", base_eval_result_large_ds),
        ("Tuned (chunk_size=250)", tuned_eval_result_large_ds),
    ],
    metrics=["accuracy", "completeness", "groundedness"],
)





print("\nBy following the 'Validate, Secure, Monitor' framework, we have successfully navigated the 'last mile'.")
print("We've moved beyond a simple prototype to a system that is measurable, secure, and ready for production challenges.")


## Additioanl Evaluation : RAGAS Metrics

Use RAGAS framework to evaluate the datasets


(a) **Context Precision:** proportion of retrieved contexts that are relevant to the query

(b) **Faithfulness:** Same as accuracy

(c) **Context Recall:** Quantifies the extent to which the relevant information

(d) **Context Relevance:** Evaluates whether the retrieved_contexts are pertinent to the user_input.

(e) **ROUGE Score:** evaluate the quality of natural language generations. It measures the overlap between the generated response and the reference text

### Define function to display eval results


In [ ]:



# Mismatch between the VertexAI library RAGAS uses and what we have used above.
#  So, new definitions needed


import pandas as pd
import plotly.graph_objects as go
from IPython.display import HTML, Markdown, display


def display_eval_report(eval_result, metrics=None):
    """Display the evaluation results."""

    # title, summary_metrics, report_df = eval_result
    title="Report"
    summary_metrics=eval_result.summary_metrics
    report_df=eval_result.metrics_table

    metrics_df = pd.DataFrame.from_dict(summary_metrics, orient="index").T
    if metrics:
        metrics_df = metrics_df.filter(
            [
                metric
                for metric in metrics_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )
        report_df = report_df.filter(
            [
                metric
                for metric in report_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    # Display the title with Markdown for emphasis
    display(Markdown(f"## {title}"))

    # Display the metrics DataFrame
    display(Markdown("### Summary Metrics"))
    display(metrics_df)

    # Display the detailed report DataFrame
    display(Markdown("### Report Metrics"))
    display(report_df)


def plot_radar_plot(eval_results, max_score=5, metrics=None):
    fig = go.Figure()

    for eval_result in eval_results:
        title="Report"
        summary_metrics=eval_result.summary_metrics
        report_df=eval_result.metrics_table

        if metrics:
            summary_metrics = {
                k: summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric in k for selected_metric in metrics)
            }

        fig.add_trace(
            go.Scatterpolar(
                r=list(summary_metrics.values()),
                theta=list(summary_metrics.keys()),
                fill="toself",
                name=title,
            )
        )

    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, max_score])), showlegend=True
    )

    fig.show()


def plot_bar_plot(eval_results, metrics=None):
    fig = go.Figure()
    data = []

    for eval_result in eval_results:

        title="Report"
        summary_metrics=eval_result.summary_metrics
        report_df=eval_result.metrics_table


        if metrics:
            summary_metrics = {
                k: summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric in k for selected_metric in metrics)
            }

        data.append(
            go.Bar(
                x=list(summary_metrics.keys()),
                y=list(summary_metrics.values()),
                name=title,
            )
        )

    fig = go.Figure(data=data)

    # Change the bar mode
    fig.update_layout(barmode="group")
    fig.show()

### Initialize and define metrics


In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_google_vertexai import VertexAI, VertexAIEmbeddings


evaluator_llm = LangchainLLMWrapper(VertexAI(model_name=MODEL_ID))
evaluator_embeddings = LangchainEmbeddingsWrapper(VertexAIEmbeddings(model_name=EMBEDDING_MODEL))

In [ ]:
from ragas import evaluate
from ragas.metrics import ContextPrecision, Faithfulness, RubricsScore, RougeScore, ContextRecall, ContextRelevance
from ragas.metrics import RougeScore
rouge_score = RougeScore()

helpfulness_rubrics = {
    "score1_description": "Response is useless/irrelevant, contains inaccurate/deceptive/misleading information, and/or contains harmful/offensive content. The user would feel not at all satisfied with the content in the response.",
    "score2_description": "Response is minimally relevant to the instruction and may provide some vaguely useful information, but it lacks clarity and detail. It might contain minor inaccuracies. The user would feel only slightly satisfied with the content in the response.",
    "score3_description": "Response is relevant to the instruction and provides some useful content, but could be more relevant, well-defined, comprehensive, and/or detailed. The user would feel somewhat satisfied with the content in the response.",
    "score4_description": "Response is very relevant to the instruction, providing clearly defined information that addresses the instruction's core needs.  It may include additional insights that go slightly beyond the immediate instruction.  The user would feel quite satisfied with the content in the response.",
    "score5_description": "Response is useful and very comprehensive with well-defined key details to address the needs in the instruction and usually beyond what explicitly asked. The user would feel very satisfied with the content in the response.",
}

rubrics_score = RubricsScore(name="helpfulness", rubrics=helpfulness_rubrics)
context_precision = ContextPrecision(llm=evaluator_llm)
faithfulness = Faithfulness(llm=evaluator_llm)
context_recall=ContextRecall(llm=evaluator_llm)
context_relevance=ContextRelevance(llm=evaluator_llm)
rouge_score = RougeScore()

### Prepare evaluation dataset in RAGAS format


In [ ]:
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset

n = len(eval_dataset_small)

samples_a = []

eval_dataset_small_prompt=eval_dataset_small['prompt'].tolist()
eval_dataset_small_context=eval_dataset_small['retrieved_context'].tolist()
eval_dataset_small_response=eval_dataset_small['response'].tolist()

eval_dataset_small_reference=['a massive, persistent anticyclonic storm','ice particles along with some rocky debris ','Mars','sulphuric acid']


for i in range(n):
    sample_a = SingleTurnSample(
        user_input=eval_dataset_small_prompt[i],
        retrieved_contexts=[eval_dataset_small_context[i]],
        response=eval_dataset_small_response[i],
        reference=eval_dataset_small_reference[i]
    )


    samples_a.append(sample_a)

ragas_eval_dataset_a = EvaluationDataset(samples=samples_a)
ragas_eval_dataset_a.to_pandas()



### *Evaluate*


In [ ]:
from ragas import evaluate

ragas_metrics = [
    context_precision,
    faithfulness,
    rouge_score,
    rubrics_score,
    context_recall,
    context_relevance
    ]

ragas_result_rag_a = evaluate(
    dataset=ragas_eval_dataset_a, metrics=ragas_metrics, llm=evaluator_llm
)

In [ ]:
from vertexai.evaluation import EvalResult

result_rag_a = EvalResult(
    summary_metrics=ragas_result_rag_a._repr_dict,
    metrics_table=ragas_result_rag_a.to_pandas(),
)

result_rag_a.summary_metrics

### Display Results


In [ ]:

metrics_list=["context_precision","faithfulness","rouge_score","helpfulness","context_recall","context_relevance"]
display_eval_report(result_rag_a, metrics=metrics_list)
plot_radar_plot([result_rag_a], metrics=metrics_list)
plot_bar_plot([result_rag_a], metrics=metrics_list)


